In [9]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [10]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [11]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
X, Y, Z = data_handling.preprocess_data(cell_images, cell_labels, cell_types)

print([cell_images.shape, len(cell_labels), len(cell_types)])
print([X.shape, len(Y), len(Z)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

@interact(n = (0, X.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(X[n], cmap = 'gray')
    axs.set_title(f'Class: {Y[n]} - Type: {Z[n]}')
    return None

[(2624, 300, 300), 2624, 2624]
[(2624, 300, 300, 3), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [12]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * X.shape[0])
X_train, Y_train, Z_train = X[: train_test_split], Y[: train_test_split], Z[: train_test_split]
X_test, Y_test, Z_test = X[train_test_split : ], Y[train_test_split : ], Z[train_test_split : ]

In [13]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('adam')
vgg19 = \
    model_development.vgg19(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = \
    model_development.train_classification_model(training_phase = 1, model = vgg19, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '20221003_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 5, batch_size = 4, validation_split_ratio = 0.2)

Epoch 1/10
446/446 [==============================] - ETA: 0s - loss: 0.6880 - accuracy: 0.5757 - precision: 0.4864 - recall: 0.3118
Epoch 1: val_accuracy improved from -inf to 0.63453, saving model to ../models/weights\VGG19_20221003_fs.hdf5
446/446 [==============================] - 21s 42ms/step - loss: 0.6880 - accuracy: 0.5757 - precision: 0.4864 - recall: 0.3118 - val_loss: 0.6741 - val_accuracy: 0.6345 - val_precision: 0.6410 - val_recall: 0.4831
Epoch 2/10
445/446 [============================>.] - ETA: 0s - loss: 0.6757 - accuracy: 0.6000 - precision: 0.5320 - recall: 0.3575
Epoch 2: val_accuracy did not improve from 0.63453
446/446 [==============================] - 18s 41ms/step - loss: 0.6749 - accuracy: 0.6009 - precision: 0.5320 - recall: 0.3575 - val_loss: 0.6971 - val_accuracy: 0.5359 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
445/446 [============================>.] - ETA: 0s - loss: 0.6518 - accuracy: 0.6337 - precision: 0.5925 - recall: 0.3846
Ep

In [14]:
##
# Train classification model (second phase):
#
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/VGG19_20221003_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '20221003_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 5, batch_size = 4, validation_split_ratio = 0.2)

Epoch 1/10
446/446 [==============================] - ETA: 0s - loss: 0.6253 - accuracy: 0.6715 - precision: 0.6804 - recall: 0.4005
Epoch 1: val_accuracy improved from -inf to 0.69058, saving model to ../models/weights\VGG19_20221003_ss.hdf5


ValueError: Unable to create dataset (name already exists)